In [2]:
from pybkit.amo.atom import Yb171

from scipy import constants
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sympy.physics.wigner import wigner_6j
from sympy.physics.quantum.cg import CG

## Utility functions

In [3]:
def mF_arr(F):
    return [F - i for i in range(int(2*F + 1))]

def matrix_element_prefactor(F1, mF1, F2, mF2, q):
    return (-1)**(F2 - F1 + mF2 - mF1) * np.sqrt((2 * F1 + 1) / (2 * F2 + 1)) * CG(F1, mF1, 1, q, F2, mF2).doit()

def J_to_F_rme_prefactor(J1, J2, F1, F2, I=1/2):
    return float(wigner_6j(J1, J2, 1, F2, F1, I)) * (-1)**(F2 + J1 + 1 + I) * np.sqrt((2 * F2 + 1) * (2 * J1 + 1))

## 6s6p 3P2 --> 6s6d 3D2 

In [5]:
yb = Yb171(B_field=5)
level1 = yb.get_fine_level('6s6p 3P2')
level2 = yb.get_fine_level('6s6d 3D2')
yb.get_reduced_matrix_element(level1, level2)

0.72

In [7]:
""" < J || d || J' > reduced matrix elements """
# estimate from 3D2 lifetime + branching ratio to 3P1, 3P2
omega12 = 2 * np.pi * 654.927 * 1e12
omega22 = 2 * np.pi * 603.411 * 1e12
gamma = 1 / 24e-9
prefactor = 3 * np.pi * constants.epsilon_0 * constants.hbar * constants.c**3
dipole_moment_si = constants.physical_constants['Bohr radius'][0] * constants.e
rme = np.sqrt(gamma * prefactor / (0.6 * omega12**3 + (0.447**2 * omega22**3)))
J_rme12 = rme / dipole_moment_si # 3P1 -> 3D2 r.m.e
J_rme22 = J_rme12 * 0.447  # 3P2 -> 3D2 r.m.e.
# print(J_rme12)
# print(J_rme22)
J_rme_df = pd.DataFrame([
    {'J1': 1, 'J2': 2, 'rme': J_rme12},
    {'J1': 2, 'J2': 2, 'rme': J_rme22},
])

""" < F || d || F' > prefactors for reduced matrix elements """
J_rme = J_rme_df.loc[(J_rme_df['J1'] == 2) & (J_rme_df['J2'] == 2), 'rme'].iloc[0]
F_rme_df = pd.DataFrame([
    {'F1': 3/2, 'F2': 3/2, 'rme': J_to_F_rme_prefactor(2, 2, 3/2, 3/2) * J_rme},
    {'F1': 3/2, 'F2': 5/2, 'rme': J_to_F_rme_prefactor(2, 2, 5/2, 3/2) * J_rme},
    {'F1': 5/2, 'F2': 3/2, 'rme': J_to_F_rme_prefactor(2, 2, 3/2, 5/2) * J_rme},
    {'F1': 5/2, 'F2': 5/2, 'rme': J_to_F_rme_prefactor(2, 2, 5/2, 5/2) * J_rme}
])

""" < F mF | d | F' mF' > prefactors for matrix elements """
Fs = [3/2, 5/2]
matrix_elements = []
for F1 in Fs:
    for F2 in Fs:
        for mF1 in mF_arr(F1):
            for mF2 in mF_arr(F2):
                for q in [-1, 0, 1]:
                    coeff = float(matrix_element_prefactor(F1, mF1, F2, mF2, q))
                    F_rme = F_rme_df.loc[(F_rme_df['F1'] == F1) & (F_rme_df['F2'] == F2), 'rme'].iloc[0]
                    if coeff != 0:
                        matrix_element = {
                            'F1': F1,
                            'F2': F2,
                            'mF1': mF1,
                            'mF2': mF2,
                            'q': q,
                            'coeff': coeff,
                            'matrix element': F_rme * coeff}
                        # print(F_rme * coeff)
                        level1 = yb.get_hyperfine_level(f'6s6p 3P2 F={F1} mF={mF1}')
                        level2 = yb.get_hyperfine_level(f'6s6d 3D2 F={F2} mF={mF2}')
                        # print(yb.get_matrix_element(level1, level2))
                        matrix_elements.append(matrix_element)
                        
df = pd.DataFrame(matrix_elements)
# df

## 6s6p 3P0 --> 6s7s 3S1 

In [12]:
""" < J || d || J' > reduced matrix element """
J_rme = 1.98  # atomic units, from Porsev et al. 1999

""" < F || d || F' > prefactors for reduced matrix elements """
F_rme_df = pd.DataFrame([
    {'F1': 1/2, 'F2': 1/2, 'rme': J_to_F_rme_prefactor(0, 1, 1/2, 1/2) * J_rme},
    {'F1': 1/2, 'F2': 3/2, 'rme': J_to_F_rme_prefactor(0, 1, 1/2, 3/2) * J_rme},
])

""" < F mF | d | F' mF' > prefactors for matrix elements """
F1s = [1/2]
F2s = [1/2, 3/2]
matrix_elements = []
for F1 in F1s:
    for F2 in F2s:
        for mF1 in mF_arr(F1):
            for mF2 in mF_arr(F2):
                for q in [-1, 0, 1]:
                    coeff = float(matrix_element_prefactor(F1, mF1, F2, mF2, q))
                    F_rme = F_rme_df.loc[(F_rme_df['F1'] == F1) & (F_rme_df['F2'] == F2), 'rme'].iloc[0]
                    if coeff != 0:
                        matrix_element = {
                            'F1': F1,
                            'F2': F2,
                            'mF1': mF1,
                            'mF2': mF2,
                            'q': q,
                            'coeff': coeff,
                            'matrix element': F_rme * coeff}
                        # print(F_rme * coeff)
                        level1 = yb.get_hyperfine_level(f'6s6p 3P0 F={F1} mF={mF1}')
                        level2 = yb.get_hyperfine_level(f'6s7s 3S1 F={F2} mF={mF2}')
                        # print(yb.get_matrix_element(level1, level2))
                        matrix_elements.append(matrix_element)
                        
df = pd.DataFrame(matrix_elements)
df[df['q'] == 1]
# df

,F1,F2,mF1,mF2,q,coeff,matrix element
2,0.5,0.5,-0.5,0.5,1,0.816497,0.933381
4,0.5,1.5,0.5,1.5,1,0.707107,1.143154
7,0.5,1.5,-0.5,0.5,1,0.408248,0.660000


## 6s6p 3P2 --> 6s7s 3S1 

In [6]:
level1 = yb.get_fine_level('6s6p 3P2')
level2 = yb.get_fine_level('6s7s 3S1')
yb.get_reduced_matrix_element(level1, level2)
5.05 / np.sqrt(2 * 2 + 1)

2.2584286572747874

In [11]:
""" < J || d || J' > reduced matrix element """
J_rme = 5.05 / np.sqrt(2 * 2 + 1)  # atomic units, from Porsev et al. 1999

""" < F || d || F' > prefactors for reduced matrix elements """
F_rme_df = pd.DataFrame([
    {'F1': 3/2, 'F2': 1/2, 'rme': J_to_F_rme_prefactor(2, 1, 3/2, 1/2) * J_rme},
    {'F1': 3/2, 'F2': 3/2, 'rme': J_to_F_rme_prefactor(2, 1, 3/2, 3/2) * J_rme},
    {'F1': 5/2, 'F2': 1/2, 'rme': J_to_F_rme_prefactor(2, 1, 5/2, 1/2) * J_rme},
    {'F1': 5/2, 'F2': 3/2, 'rme': J_to_F_rme_prefactor(2, 1, 5/2, 3/2) * J_rme},
])

""" < F mF | d | F' mF' > prefactors for matrix elements """
F1s = [3/2, 5/2]
F2s = [1/2, 3/2]
matrix_elements = []
for F1 in F1s:
    for F2 in F2s:
        for mF1 in mF_arr(F1):
            for mF2 in mF_arr(F2):
                for q in [-1, 0, 1]:
                    coeff = float(matrix_element_prefactor(F1, mF1, F2, mF2, q))
                    F_rme = F_rme_df.loc[(F_rme_df['F1'] == F1) & (F_rme_df['F2'] == F2), 'rme'].iloc[0]
                    if coeff != 0:
                        matrix_element = {
                            'F1': F1,
                            'F2': F2,
                            'mF1': mF1,
                            'mF2': mF2,
                            'q': q,
                            'coeff': coeff,
                            'matrix element': F_rme * coeff}
                        # print(F_rme * coeff)
                        level1 = yb.get_hyperfine_level(f'6s6p 3P2 F={F1} mF={mF1}')
                        level2 = yb.get_hyperfine_level(f'6s7s 3S1 F={F2} mF={mF2}')
                        # print(yb.get_matrix_element(level1, level2))
                        matrix_elements.append(matrix_element)
                        
df = pd.DataFrame(matrix_elements)
df[df['q'] == -1]

,F1,F2,mF1,mF2,q,coeff,matrix element
0,1.5,0.5,1.5,0.5,-1,1.000000,2.061654
2,1.5,0.5,0.5,-0.5,-1,0.577350,1.190296
7,1.5,1.5,1.5,0.5,-1,-0.632456,0.583124
10,1.5,1.5,0.5,-0.5,-1,-0.730297,0.673333
13,1.5,1.5,-0.5,-1.5,-1,-0.632456,0.583124
16,2.5,1.5,2.5,1.5,-1,1.000000,2.258429
18,2.5,1.5,1.5,0.5,-1,0.774597,1.749371
21,2.5,1.5,0.5,-0.5,-1,0.547723,1.236992
24,2.5,1.5,-0.5,-1.5,-1,0.316228,0.714178


In [ ]:
1.749 / 2.258

In [ ]:
np.sqrt(3/5)

In [ ]:
def calculate_E_field(power, beam_waist):
    intensity = 2 * power / (np.pi * beam_waist**2)
    E_field = np.sqrt(2 * intensity / (constants.c * constants.epsilon_0))
    return E_field

def calculate_rabi_frequency(dipole_matrix_element, E_field):
    return np.abs(dipole_matrix_element * E_field / constants.hbar)

In [ ]:
yb = Yb171(B_field=5e-4)
dipole_moment_si = constants.physical_constants['Bohr radius'][0] * constants.e
dipole_matrix_element = yb.get_reduced_matrix_element('6s6p 3P2', '6s6d 3D2') * dipole_moment_si
print(f"{dipole_matrix_element = }")

In [ ]:
power = 1e-3  # laser power [watts]
beam_waist = 500e-9  # beam waist [meters]

E_field = calculate_E_field(power, beam_waist)
rabi_freq = calculate_rabi_frequency(dipole_matrix_element, E_field)
print(1e-6 * rabi_freq / (2 * np.pi))

In [ ]:
power_649 = 100e-3  # laser power [watts]
beam_waist_649 = 500e-6  # beam waist [meters]
dipole_moment_si = constants.physical_constants['Bohr radius'][0] * constants.e
dipole_matrix_element_649 = 1.143154 * dipole_moment_si
E_field_649 = calculate_E_field(power_649, beam_waist_649)
rabi_freq_649 = calculate_rabi_frequency(dipole_matrix_element_649, E_field_649)
print(dipole_matrix_element_649 * np.sqrt(4 * 1e-3 / (np.pi * beam_waist_649**2 * constants.c * constants.epsilon_0)) / constants.hbar / 2 / np.pi / 1e6)

power_770 = 100e-3  # laser power [watts]
beam_waist_770 = 500e-6  # beam waist [meters]
dipole_moment_si = constants.physical_constants['Bohr radius'][0] * constants.e
dipole_matrix_element_770 = 2.258429 * dipole_moment_si
E_field_770 = calculate_E_field(power_770, beam_waist_770)
rabi_freq_770 = calculate_rabi_frequency(dipole_matrix_element_770, E_field_770)
print(dipole_matrix_element_770 * np.sqrt(4 * 1e-3 / (np.pi * beam_waist_770**2 * constants.c * constants.epsilon_0)) / constants.hbar / 2 / np.pi / 1e6)


print(rabi_freq_649 / (2 * np.pi) / 1e6)
print(rabi_freq_770 / (2 * np.pi) / 1e6)

raman_detuning = 2 * np.pi * 50e9

print((rabi_freq_649 / raman_detuning)**2)
print((rabi_freq_770 / raman_detuning)**2)

raman_rabi_freq = rabi_freq_649 * rabi_freq_770 / (2 * raman_detuning)
print(raman_rabi_freq / (2 * np.pi) / 1e6)

In [22]:
[key for key in constants.physical_constants.keys() if 'Bohr' in key]

['deuteron magn. moment to Bohr magneton ratio',
 'electron magn. moment to Bohr magneton ratio',
 'muon magn. moment to Bohr magneton ratio',
 'neutron magn. moment to Bohr magneton ratio',
 'proton magn. moment to Bohr magneton ratio',
 'shielded helion magn. moment to Bohr magneton ratio',
 'shielded proton magn. moment to Bohr magneton ratio',
 'Bohr magneton',
 'Bohr magneton in eV/T',
 'Bohr magneton in Hz/T',
 'Bohr magneton in inverse meters per tesla',
 'Bohr magneton in K/T',
 'Bohr radius',
 'deuteron mag. mom. to Bohr magneton ratio',
 'electron mag. mom. to Bohr magneton ratio',
 'muon mag. mom. to Bohr magneton ratio',
 'neutron mag. mom. to Bohr magneton ratio',
 'proton mag. mom. to Bohr magneton ratio',
 'shielded helion mag. mom. to Bohr magneton ratio',
 'shielded proton mag. mom. to Bohr magneton ratio',
 'triton mag. mom. to Bohr magneton ratio',
 'helion mag. mom. to Bohr magneton ratio',
 'Bohr magneton in inverse meter per tesla']

In [33]:
from scipy import constants 

matrix_element = 0.58 * constants.e * constants.physical_constants['Bohr radius'][0]
w0 = 500 * 1e-9
factor = np.sqrt(4 / constants.epsilon_0 / constants.c / np.pi / w0**2)
matrix_element * factor / constants.hbar / 2 / np.pi / 1e6 * np.sqrt(1e-6) 

325.0751932508856